# Day-Ahead Prices — ENTSO-E Examples

Querying and visualizing day-ahead electricity market prices.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from library import Client

client = Client()

## 1. Day-Ahead Prices — France (1 week)

In [ ]:
start = pd.Timestamp("2024-06-01", tz="Europe/Paris")
end = pd.Timestamp("2024-06-08", tz="Europe/Paris")

df_prices = client.prices.day_ahead(start, end, country="FR")
df_prices.head()

In [ ]:
fig = px.line(
    df_prices, x="timestamp", y="value",
    title="Day-Ahead Prices — France",
    labels={"value": "Price (EUR/MWh)", "timestamp": ""},
)
fig.show()

## 2. Multi-Country Price Comparison

In [ ]:
countries = {"FR": "France", "DE": "Germany", "ES": "Spain", "NL": "Netherlands"}
frames = []
for code, name in countries.items():
    df = client.prices.day_ahead(start, end, country=code)
    df["country"] = name
    frames.append(df)

df_multi = pd.concat(frames, ignore_index=True)

fig = px.line(
    df_multi, x="timestamp", y="value", color="country",
    title="Day-Ahead Prices — Multi-Country Comparison",
    labels={"value": "Price (EUR/MWh)", "timestamp": ""},
)
fig.show()

## 3. Price Distribution

In [ ]:
fig = px.box(
    df_multi, x="country", y="value", color="country",
    title="Price Distribution — Week of 1-8 Jun 2024",
    labels={"value": "Price (EUR/MWh)", "country": ""},
)
fig.show()

## 4. Hourly Price Heatmap — France (1 month)

In [ ]:
start = pd.Timestamp("2024-06-01", tz="Europe/Paris")
end = pd.Timestamp("2024-07-01", tz="Europe/Paris")

df_month = client.prices.day_ahead(start, end, country="FR")
df_month["hour"] = df_month["timestamp"].dt.hour
df_month["date"] = df_month["timestamp"].dt.date

pivot = df_month.pivot_table(index="hour", columns="date", values="value")

fig = px.imshow(
    pivot,
    title="Hourly Day-Ahead Prices — France, June 2024",
    labels=dict(x="Date", y="Hour", color="EUR/MWh"),
    aspect="auto",
    color_continuous_scale="RdYlGn_r",
)
fig.show()